In [1]:
import os
import json

from pathlib import Path
from collections import defaultdict, Counter
from itertools import chain, starmap

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [178]:
df_15 = df_15_bu.copy()
df_15[df_15.columns[-7:-1]] = df_15[df_15.columns[-6:]]
df_15.drop("Station", axis=1, inplace=True)

In [179]:
df_15.head()

,Status,Mnet_Id,Elevation,Name,Stid,Elev_Dem,Longitude,State,Restricted,Qc_Flagged,Latitude,Timezone,Id,Period_Of_Record.Start,Period_Of_Record.End,Observations.Date_Time,Observations.Relative_Humidity_Set_1,Observations.Wind_Speed_Set_1,Observations.Wind_Gust_Set_1
0,ACTIVE,231,6388,SCE Marion Ridge,SE001,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11T20:11:00Z,2021-06-09T17:50:00Z,2021-01-15T00:00:00Z,22.64,3.33,6.72
1,ACTIVE,231,6388,SCE Marion Ridge,SE001,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11T20:11:00Z,2021-06-09T17:50:00Z,2021-01-15T00:10:00Z,24.48,4.06,10.45
2,ACTIVE,231,6388,SCE Marion Ridge,SE001,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11T20:11:00Z,2021-06-09T17:50:00Z,2021-01-15T00:20:00Z,25.66,0.78,2.34
3,ACTIVE,231,6388,SCE Marion Ridge,SE001,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11T20:11:00Z,2021-06-09T17:50:00Z,2021-01-15T00:30:00Z,25.29,1.46,4.89
4,ACTIVE,231,6388,SCE Marion Ridge,SE001,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11T20:11:00Z,2021-06-09T17:50:00Z,2021-01-15T00:40:00Z,25.61,1.72,3.43


In [182]:
dfs = [df_12, df_15, df_18]

In [183]:
df = pd.concat(dfs)

In [184]:
df["Station"] = df.Stid.str.replace("SE", "").apply(lambda i: int(i))

In [195]:
df = df.rename(columns = {"Observations.Date_Time":"Time", "Observations.Relative_Humidity_Set_1":"RelHum", "Observations.Wind_Speed_Set_1":"WindSpd", "Observations.Wind_Gust_Set_1":"GustSpd"})

In [186]:
df.reset_index(drop=True, inplace=True)

In [196]:
df.to_csv("./sce.csv", index=False)

In [162]:
len(df.index)

1013946

In [276]:
sce = pd.read_csv("sce.csv", parse_dates = ["Time", "Date", "Period_Of_Record.Start", "Period_Of_Record.End"])

In [277]:
sce.dtypes

Status                                 object
Mnet_Id                                 int64
Elevation                               int64
Name                                   object
Elev_Dem                              float64
Longitude                             float64
State                                  object
Restricted                               bool
Qc_Flagged                               bool
Latitude                              float64
Timezone                               object
Id                                      int64
Period_Of_Record.Start    datetime64[ns, UTC]
Period_Of_Record.End      datetime64[ns, UTC]
Time                      datetime64[ns, UTC]
RelHum                                float64
WindSpd                               float64
GustSpd                               float64
Date                           datetime64[ns]
Station                                object
dtype: object

In [280]:
sce.Time = sce.Time.dt.tz_localize(None)

In [263]:
sce.RelHum.describe(percentiles=np.arange(0,1,.1))

count    1.013935e+06
mean     2.881012e+01
std      1.654797e+01
min      8.800000e-01
0%       8.800000e-01
10%      1.341000e+01
20%      1.609000e+01
30%      1.838000e+01
40%      2.099000e+01
50%      2.410000e+01
60%      2.781000e+01
70%      3.253000e+01
80%      3.898000e+01
90%      5.028000e+01
max      1.000000e+02
Name: RelHum, dtype: float64

In [281]:
sce.head()

,Status,Mnet_Id,Elevation,Name,Elev_Dem,Longitude,State,Restricted,Qc_Flagged,Latitude,Timezone,Id,Period_Of_Record.Start,Period_Of_Record.End,Time,RelHum,WindSpd,GustSpd,Date,Station
0,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:00:00,30.89,2.87,6.43,2021-01-12,SE001
1,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:10:00,30.83,3.27,8.26,2021-01-12,SE001
2,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:20:00,31.16,2.16,6.06,2021-01-12,SE001
3,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:30:00,32.43,1.29,4.38,2021-01-12,SE001
4,ACTIVE,231,6388,SCE Marion Ridge,6400.9,-116.73432,CA,False,False,33.76594,America/Los_Angeles,70844,2018-06-11 20:11:00+00:00,2021-06-09 17:50:00+00:00,2021-01-12 00:40:00,32.18,1.32,3.58,2021-01-12,SE001


In [282]:
sce.to_csv("./sce.csv", index=False)

In [283]:
import os

In [284]:
os.getcwd()

'/Users/tomityrrell/Documents/GitHub/psps-analysis/loaders/mesowest'